# Word2Vec Test: Euclidean Geometry

**Goal:** Test the canonical word2vec arithmetic in **Euclidean space**.

**Hypothesis:** `'woman' - 'man' + 'king' ≈ 'queen'`

**Why this matters:** This is the **classic test** of linear semantic relationships. If it works in Euclidean space, we know the raw embeddings have compositional structure. We'll compare with 07.59b (causal metric) to see if the metric preserves or destroys this structure.

**Method:**
1. Compute synthetic vector: `v = 'woman' - 'man' + 'king'`
2. Find nearest neighbors by **Euclidean distance**
3. Find nearest neighbors by **Euclidean cosine similarity**
4. Check where 'queen' ranks

**Note:** We search the **full vocabulary** (151k tokens), not just our 32k sample.

## Configuration

In [27]:
# Model
MODEL_NAME = 'Qwen/Qwen3-4B-Instruct-2507'

# Tokens for arithmetic
TOKENS = {
    'man': None,    # Will tokenize
    'woman': None,  # Will tokenize
    'king': None,   # Will tokenize
    'queen': None,  # Will tokenize (for checking)
}

# Analysis parameters
TOP_N = 20  # How many neighbors to show

print(f"Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Arithmetic: 'woman' - 'man' + 'king' = ?")
print(f"  Metric: EUCLIDEAN")
print(f"  Search space: Full vocabulary (151k tokens)")
print(f"  Top N results: {TOP_N}")

Configuration:
  Model: Qwen/Qwen3-4B-Instruct-2507
  Arithmetic: 'woman' - 'man' + 'king' = ?
  Metric: EUCLIDEAN
  Search space: Full vocabulary (151k tokens)
  Top N results: 20


## Setup

In [28]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

print("✓ Imports complete")

✓ Imports complete


## Load Model and Tokenizer

In [29]:
print("Loading model and tokenizer...\n")

# Tokenizer
print(f"Loading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Model (for full unembedding matrix)
print(f"\nLoading model (for unembedding matrix)...")
print("  This will take a minute...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map='cpu',
)

# Extract FULL unembedding matrix (all vocab)
gamma = model.lm_head.weight.data.to(torch.float32).cpu()  # [vocab_size, hidden_dim]

print(f"\n✓ Model loaded")
print(f"  Vocab size: {tokenizer.vocab_size:,}")
print(f"  Unembedding matrix shape: {gamma.shape}")
print(f"  Memory: {gamma.element_size() * gamma.nelement() / 1e9:.2f} GB")

Loading model and tokenizer...

Loading tokenizer from Qwen/Qwen3-4B-Instruct-2507...

Loading model (for unembedding matrix)...
  This will take a minute...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


✓ Model loaded
  Vocab size: 151,643
  Unembedding matrix shape: torch.Size([151936, 2560])
  Memory: 1.56 GB


## Tokenize and Verify

In [30]:
print("Tokenizing target words...\n")

words = ['man', 'woman', 'king', 'queen']
all_single_tokens = True

for word in words:
    tokens = tokenizer.encode(word, add_special_tokens=False)
    
    if len(tokens) == 1:
        token_id = tokens[0]
        TOKENS[word] = token_id
        text = tokenizer.decode([token_id])
        print(f"✓ '{word}' → token {token_id}: '{text}'")
    else:
        print(f"✗ '{word}' → {len(tokens)} tokens: {tokens}")
        all_single_tokens = False

if not all_single_tokens:
    print("\n⚠️  Not all words are single tokens!")
    print("    This may affect results, but we'll proceed anyway.")
else:
    print("\n✓ All words are single tokens")

Tokenizing target words...

✓ 'man' → token 1515: 'man'
✓ 'woman' → token 22028: 'woman'
✓ 'king' → token 10566: 'king'
✓ 'queen' → token 93114: 'queen'

✓ All words are single tokens


## Compute Synthetic Vector: 'woman' - 'man' + 'king'

In [31]:
print("\nComputing synthetic vector...\n")

# Get embeddings
v_man = gamma[TOKENS['man']]
v_woman = gamma[TOKENS['woman']]
v_king = gamma[TOKENS['king']]

# Arithmetic
v_synthetic = v_woman - v_man + v_king

print(f"v_synthetic = 'woman' - 'man' + 'king'")
print(f"\nSynthetic vector properties:")
print(f"  Shape: {v_synthetic.shape}")
print(f"  Euclidean norm: {torch.norm(v_synthetic).item():.2f}")


Computing synthetic vector...

v_synthetic = 'woman' - 'man' + 'king'

Synthetic vector properties:
  Shape: torch.Size([2560])
  Euclidean norm: 1.81


## Direct Vector Comparison: Are the Transformations Parallel?

**Simple test:** Do `woman - man` and `queen - king` point in the same direction?

If word2vec arithmetic works, these should be **parallel** (or at least highly aligned).

In [32]:
print("\n" + "=" * 80)
print("DIRECT VECTOR COMPARISON (EUCLIDEAN)")
print("=" * 80)

# Get queen vector
v_queen = gamma[TOKENS['queen']]

# Compute the two displacement vectors
v_gender_shift = v_woman - v_man      # man → woman
v_royalty_shift = v_queen - v_king    # king → queen

# Euclidean properties
norm_gender = torch.norm(v_gender_shift).item()
norm_royalty = torch.norm(v_royalty_shift).item()

# Cosine similarity and angle
dot_product = torch.dot(v_gender_shift, v_royalty_shift).item()
cosine_similarity = dot_product / (norm_gender * norm_royalty)
angle_rad = torch.acos(torch.clamp(torch.tensor(cosine_similarity), -1.0, 1.0))
angle_deg = torch.rad2deg(angle_rad).item()

print(f"\n(woman - man) properties:")
print(f"  Euclidean norm: {norm_gender:.4f}")

print(f"\n(queen - king) properties:")
print(f"  Euclidean norm: {norm_royalty:.4f}")

print(f"\nAlignment between the two vectors:")
print(f"  Dot product: {dot_product:.4f}")
print(f"  Cosine similarity: {cosine_similarity:.4f}")
print(f"  Angle: {angle_deg:.2f}°")

print(f"\n" + "=" * 80)
print("INTERPRETATION:")
print("=" * 80)

if abs(cosine_similarity) > 0.9:
    print(f"✅ Vectors are HIGHLY ALIGNED (cos={cosine_similarity:.3f})")
    print(f"   Word2vec arithmetic should work!")
elif abs(cosine_similarity) > 0.7:
    print(f"✓ Vectors are moderately aligned (cos={cosine_similarity:.3f})")
    print(f"  Some linear structure exists")
elif abs(cosine_similarity) > 0.3:
    print(f"⚠️  Vectors are weakly aligned (cos={cosine_similarity:.3f})")
    print(f"   Minimal linear relationship")
else:
    print(f"❌ Vectors are nearly ORTHOGONAL (cos={cosine_similarity:.3f}, angle={angle_deg:.1f}°)")
    print(f"   No meaningful linear relationship")
    print(f"   Word2vec arithmetic will NOT work")

print(f"\nFor reference:")
print(f"  • Parallel vectors: cos=1.0, angle=0°")
print(f"  • Orthogonal vectors: cos=0.0, angle=90°")
print(f"  • Opposite vectors: cos=-1.0, angle=180°")


DIRECT VECTOR COMPARISON (EUCLIDEAN)

(woman - man) properties:
  Euclidean norm: 1.3019

(queen - king) properties:
  Euclidean norm: 1.4175

Alignment between the two vectors:
  Dot product: 0.2379
  Cosine similarity: 0.1289
  Angle: 82.59°

INTERPRETATION:
❌ Vectors are nearly ORTHOGONAL (cos=0.129, angle=82.6°)
   No meaningful linear relationship
   Word2vec arithmetic will NOT work

For reference:
  • Parallel vectors: cos=1.0, angle=0°
  • Orthogonal vectors: cos=0.0, angle=90°
  • Opposite vectors: cos=-1.0, angle=180°


## Find Nearest Neighbors by Euclidean Distance

## Testing Scaled Vector Arithmetic

**Question:** Is the failure due to **magnitude** or **direction**?

**Hypothesis:** Maybe `queen = king + α * (woman - man)` for some optimal α ≠ 1

**Method:** Solve for optimal α via least-squares projection:

```
α = (queen - king) · (woman - man) / ||woman - man||²
```

If α ≈ 0, the vectors are **orthogonal** (no relationship).

If α is reasonable but residual is large, the direction is right but imperfect.

If α ≈ 1 and residual is small, we just needed scaling!

In [22]:
print("\n" + "=" * 80)
print("SCALED VECTOR ARITHMETIC TEST (EUCLIDEAN)")
print("=" * 80)

# Define vectors
v_gender_shift = v_woman - v_man  # The gender transformation vector
v_queen = gamma[TOKENS['queen']]
v_target_displacement = v_queen - v_king  # What we actually need to add to king

# Solve for optimal α: queen = king + α * (woman - man)
# α = (queen - king) · (woman - man) / ||woman - man||²
numerator = torch.dot(v_target_displacement, v_gender_shift).item()
denominator = torch.dot(v_gender_shift, v_gender_shift).item()
alpha_optimal = numerator / denominator

print(f"\nOptimal scaling factor α = {alpha_optimal:.6f}")
print(f"\nInterpretation:")
if abs(alpha_optimal) < 0.01:
    print(f"  α ≈ 0 → Gender shift is ORTHOGONAL to king→queen displacement")
    print(f"  The vectors have no meaningful relationship")
elif 0.8 <= alpha_optimal <= 1.2:
    print(f"  α ≈ 1 → Gender shift aligns well with king→queen displacement!")
    print(f"  Word2vec arithmetic works (just needed right magnitude)")
else:
    print(f"  α = {alpha_optimal:.2f} → Vectors are aligned but need rescaling")
    print(f"  Partial compositional structure")

# Construct scaled synthetic vector
v_scaled = v_king + alpha_optimal * v_gender_shift

# === EUCLIDEAN DISTANCE ===
euclidean_dist_synthetic_to_queen = torch.norm(v_synthetic - v_queen).item()
euclidean_dist_scaled_to_queen = torch.norm(v_scaled - v_queen).item()

# === EUCLIDEAN ANGLE ===
# Compute cosine and convert to degrees
cos_synthetic_queen = torch.dot(v_synthetic, v_queen) / (torch.norm(v_synthetic) * torch.norm(v_queen))
angle_synthetic_queen_rad = torch.acos(torch.clamp(cos_synthetic_queen, -1.0, 1.0))
angle_synthetic_queen_deg = torch.rad2deg(angle_synthetic_queen_rad).item()

cos_scaled_queen = torch.dot(v_scaled, v_queen) / (torch.norm(v_scaled) * torch.norm(v_queen))
angle_scaled_queen_rad = torch.acos(torch.clamp(cos_scaled_queen, -1.0, 1.0))
angle_scaled_queen_deg = torch.rad2deg(angle_scaled_queen_rad).item()

print(f"\n{'Method':<30} {'Distance':<15} {'Angle (degrees)':<15}")
print("=" * 60)
print(f"{'Original (α=1.0)':<30} {euclidean_dist_synthetic_to_queen:<15.4f} {angle_synthetic_queen_deg:<15.2f}")
print(f"{'Scaled (α=' + f'{alpha_optimal:.4f})':<30} {euclidean_dist_scaled_to_queen:<15.4f} {angle_scaled_queen_deg:<15.2f}")

dist_improvement = (1 - euclidean_dist_scaled_to_queen / euclidean_dist_synthetic_to_queen) * 100
angle_improvement = (1 - angle_scaled_queen_deg / angle_synthetic_queen_deg) * 100

print(f"\nDistance improvement: {dist_improvement:.1f}%")
print(f"Angular improvement: {angle_improvement:.1f}%")

if dist_improvement > 50:
    print("  → LARGE distance improvement! The issue was magnitude, not direction")
elif dist_improvement > 10:
    print("  → Moderate distance improvement. Direction is partially correct")
else:
    print("  → Minimal distance improvement. The vectors are not well aligned")


SCALED VECTOR ARITHMETIC TEST (EUCLIDEAN)

Optimal scaling factor α = 0.140366

Interpretation:
  α = 0.14 → Vectors are aligned but need rescaling
  Partial compositional structure

Method                         Distance        Angle (degrees)
Original (α=1.0)               1.7968          70.84          
Scaled (α=0.1404)              1.4057          71.69          

Distance improvement: 21.8%
Angular improvement: -1.2%
  → Moderate distance improvement. Direction is partially correct


In [23]:
print("\nComputing Euclidean distances to ALL tokens in vocabulary...\n")
print("  (This may take a minute for 151k tokens...)\n")

# Compute Euclidean distances: ||u - v||_2
vocab_size = gamma.shape[0]
euclidean_distances = torch.norm(gamma - v_synthetic, dim=1).numpy()

# Sort by distance (ascending)
sorted_indices = np.argsort(euclidean_distances)
top_indices = sorted_indices[:TOP_N]

print(f"Top {TOP_N} tokens by EUCLIDEAN DISTANCE:")
print(f"{'Rank':<6} {'Distance':<12} {'Token ID':<10} {'Text':<40}")
print("=" * 80)

for rank, idx in enumerate(top_indices, 1):
    dist = euclidean_distances[idx]
    text = tokenizer.decode([idx])
    print(f"{rank:<6} {dist:<12.2f} {idx:<10} {text:<40}")

print("\n" + "=" * 80)


Computing Euclidean distances to ALL tokens in vocabulary...

  (This may take a minute for 151k tokens...)

Top 20 tokens by EUCLIDEAN DISTANCE:
Rank   Distance     Token ID   Text                                    
1      1.30         10566      king                                    
2      1.60         22028      woman                                   
3      1.69         33555      King                                    
4      1.71         11477       king                                   
5      1.72         73811       KING                                   
6      1.73         64662      women                                   
7      1.74         148785     𝙠                                       
8      1.74         151607     ﱊ                                       
9      1.74         150876     끅                                       
10     1.74         149446     ﳈ                                       
11     1.74         151273     𒄷                             

## Find Nearest Neighbors by Euclidean Cosine Similarity

In [24]:
print("\nComputing Euclidean cosine similarities to ALL tokens...\n")

# Compute cosine similarities
norms = torch.norm(gamma, dim=1)
norm_synthetic = torch.norm(v_synthetic)
dot_products = gamma @ v_synthetic
cosine_sims = (dot_products / (norms * norm_synthetic)).numpy()

# Sort by cosine (descending)
sorted_indices_cos = np.argsort(-cosine_sims)
top_indices_cos = sorted_indices_cos[:TOP_N]

print(f"Top {TOP_N} tokens by EUCLIDEAN COSINE SIMILARITY:")
print(f"{'Rank':<6} {'Cosine':<12} {'Distance':<12} {'Token ID':<10} {'Text':<40}")
print("=" * 80)

for rank, idx in enumerate(top_indices_cos, 1):
    cos_sim = cosine_sims[idx]
    dist = euclidean_distances[idx]
    text = tokenizer.decode([idx])
    print(f"{rank:<6} {cos_sim:<12.4f} {dist:<12.2f} {idx:<10} {text:<40}")

print("\n" + "=" * 80)


Computing Euclidean cosine similarities to ALL tokens...

Top 20 tokens by EUCLIDEAN COSINE SIMILARITY:
Rank   Cosine       Distance     Token ID   Text                                    
1      0.6945       1.30         10566      king                                    
2      0.4840       1.60         22028      woman                                   
3      0.4002       1.69         33555      King                                    
4      0.3839       1.72         73811       KING                                   
5      0.3806       1.71         11477       king                                   
6      0.3771       1.73         64662      women                                   
7      0.3472       1.75         27906       queen                                  
8      0.3332       1.79         95049      Woman                                   
9      0.3283       1.80         93114      queen                                   
10     0.3276       1.77         6210        

## Check for 'queen' Specifically

In [25]:
if TOKENS['queen'] is not None:
    idx_queen = TOKENS['queen']
    
    dist_to_queen = euclidean_distances[idx_queen]
    cos_to_queen = cosine_sims[idx_queen]
    
    # Find rank
    rank_by_distance = np.where(sorted_indices == idx_queen)[0][0] + 1
    rank_by_cosine = np.where(sorted_indices_cos == idx_queen)[0][0] + 1
    
    print("\n" + "=" * 80)
    print(f"CHECKING FOR 'queen' (token {idx_queen}):")
    print("=" * 80)
    print(f"\nEuclidean distance: {dist_to_queen:.2f} (rank {rank_by_distance})")
    print(f"Euclidean cosine: {cos_to_queen:.4f} (rank {rank_by_cosine})")
    
    if rank_by_distance <= 5:
        print(f"\n🎉🎉🎉 'queen' is in the TOP 5 by Euclidean distance!")
        print(f"         WORD2VEC MAGIC WORKS IN EUCLIDEAN SPACE!")
    elif rank_by_distance <= 10:
        print(f"\n🎉 'queen' is in the TOP 10 by Euclidean distance!")
        print(f"    Linear semantics confirmed!")
    elif rank_by_distance <= 20:
        print(f"\n✓ 'queen' is in the top 20 by Euclidean distance")
        print(f"  Moderate evidence for linear semantics")
    elif rank_by_distance <= 100:
        print(f"\n'queen' is rank {rank_by_distance} by Euclidean distance")
        print(f"  Weak signal, but detectable")
    else:
        print(f"\n❌ 'queen' is rank {rank_by_distance} by Euclidean distance")
        print(f"   Word2vec arithmetic does NOT work in this space")
    
    if rank_by_cosine <= 5:
        print(f"\n🎉🎉🎉 'queen' is in the TOP 5 by Euclidean cosine!")
    elif rank_by_cosine <= 10:
        print(f"🎉 'queen' is in the TOP 10 by Euclidean cosine!")
    elif rank_by_cosine <= 20:
        print(f"✓ 'queen' is in the top 20 by Euclidean cosine")
    elif rank_by_cosine <= 100:
        print(f"'queen' is rank {rank_by_cosine} by Euclidean cosine")
    else:
        print(f"❌ 'queen' is rank {rank_by_cosine} by Euclidean cosine")
else:
    print("\n⚠️  Token 'queen' not found or not a single token")


CHECKING FOR 'queen' (token 93114):

Euclidean distance: 1.80 (rank 4496)
Euclidean cosine: 0.3283 (rank 9)

❌ 'queen' is rank 4496 by Euclidean distance
   Word2vec arithmetic does NOT work in this space
🎉 'queen' is in the TOP 10 by Euclidean cosine!


## Overlap Analysis

In [26]:
print("\n" + "=" * 80)
print("OVERLAP ANALYSIS")
print("=" * 80)

# Find tokens that appear in BOTH top N lists
top_by_distance = set(sorted_indices[:TOP_N])
top_by_cosine = set(sorted_indices_cos[:TOP_N])

overlap = top_by_distance & top_by_cosine

print(f"\nTokens in top {TOP_N} by BOTH metrics:")
print(f"  Count: {len(overlap)}")

if overlap:
    print(f"\nTokens appearing in both lists:")
    for tid in sorted(overlap):
        text = tokenizer.decode([tid])
        dist = euclidean_distances[tid]
        cos = cosine_sims[tid]
        print(f"  • {tid}: '{text}' (distance={dist:.2f}, cosine={cos:.4f})")


OVERLAP ANALYSIS

Tokens in top 20 by BOTH metrics:
  Count: 6

Tokens appearing in both lists:
  • 10566: 'king' (distance=1.30, cosine=0.6945)
  • 11477: ' king' (distance=1.71, cosine=0.3806)
  • 22028: 'woman' (distance=1.60, cosine=0.4840)
  • 33555: 'King' (distance=1.69, cosine=0.4002)
  • 64662: 'women' (distance=1.73, cosine=0.3771)
  • 73811: ' KING' (distance=1.72, cosine=0.3839)


## Summary

**What we tested:** The classic word2vec arithmetic `'woman' - 'man' + 'king' ≈ 'queen'` in **Euclidean geometry**.

**Why this matters:** If this works, we know that:
1. The raw unembedding matrix has linear semantic structure
2. Compositional semantics are preserved in this LLM
3. Word2vec magic transfers to modern transformers

**Next step:** Compare with 07.59b (causal metric) to see if the metric preserves or destroys this structure.